In [1]:
import pandas as pd
import numpy as np
import glob
import os
import random
import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import datetime
import tensorflow.keras as keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, LSTM, Flatten, BatchNormalization, Dropout
from keras.utils import to_categorical
import tensorflow as tf
# Load the TensorBoard notebook extension
%load_ext tensorboard

Using TensorFlow backend.


In [2]:
countriesOfInterest = ["HK", "J P", 'ZA', 'TN', 'TR', 'GB', 'MX', 'US', 'CO', 'EC', 'AU', 'NZ']
countriesOfInterest = ["ZA", "EG", "TW", "JP", "DK", "FI", "US", "CA", "AU", "NZ", "BE", "CO"]
train_n = 500
val_n = 20
Category = "Country"
w_length = 300
enc = OneHotEncoder()
enc.fit(np.array(countriesOfInterest).reshape(-1, 1))

OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='error', sparse=True)

In [3]:
def split(X, cat):
    X = X.reset_index()
    new_pos = list(X.track_id.index[X.track_id.shift(1) != X.track_id]) # indices where the song changes
    new_pos.append(max(X.track_id.index) + 1) # add a new index to know where the last song ends
    split_pos = []
    for i in range(len(new_pos)-1):
        split_pos = split_pos + list(range(new_pos[i], new_pos[i+1], w_length))
    split_pos = split_pos[1:]
    us_train = np.split(X.iloc[:,:24].to_numpy(), split_pos)
    labs = np.split(X[Category].to_numpy(), split_pos)
    # drop the short sequences
    short_seqs = []
    temp = [] 
    labels = []
    for i, value in enumerate(us_train):
        if value.shape[0] == w_length:
            temp.append(value)
            labels.append(labs[i][0])
    us_train = temp
    return np.stack(us_train), labels

In [4]:
def splitSeconds(n, country, t):
    data = pickle.load( open( "Raw Track Data\\" + country + "_" + t + ".p", "rb" ) )
    tracks = data.track_id.unique()
    tracks = np.random.choice(tracks, size=n, replace=True)
    samples = []
    for track in tracks:
        try:
            trackFeats = data[data.track_id == track]
            FeatsLen =  trackFeats.shape[0]
            #ind = np.random.choice(trackFeats.index[:-5], size=1, replace=True)
            ind = random.randrange(1, FeatsLen - 10)
            example = np.empty((0,24))
            while len(example) < w_length:
                feats = np.array(trackFeats.iloc[ind].loc["p1":"t12"])
                reps = int(trackFeats.iloc[ind].loc["duration"] * 10)
                if reps > 0:
                    hold = np.tile(feats, reps).reshape(reps, -1)
                    example = np.append(example, hold, axis = 0)
                ind = ind + 1
              #  example = np.array(example)
                example = example[-w_length:]
            samples = samples + [example]
        except:
            continue
    samples = np.array(samples)
    return samples, np.repeat(np.array([country]), samples.shape[0])

In [5]:
def getSamples(train_n, val_n):
    train = pd.DataFrame()
    train_labels = pd.DataFrame()
    val = pd.DataFrame()
    val_labels = pd.DataFrame()
    train_x = []
    train_labels = []
    val_x = []
    val_labels = []
    for country in countriesOfInterest:
        print("getting",country)
        x1, y1 = splitSeconds(train_n, country, "train")
        x2, y2 = splitSeconds(val_n, country, "val")
        train_x = train_x + x1.tolist()
        train_labels = train_labels + y1.tolist()
        val_x = val_x + x2.tolist()
        val_labels = val_labels + y2.tolist()
    train_x = np.array(train_x)
    train_labels = np.array(train_labels)
    val_x = np.array(val_x)
    val_labels = np.array(val_labels)
    class_weights = class_weight.compute_class_weight('balanced',
                                                     np.unique(train_labels),
                                                     list(train_labels))
    train_labels = enc.transform(np.array(train_labels).reshape(-1,1)).toarray()
    val_labels = enc.transform(np.array(val_labels).reshape(-1,1)).toarray()
    return train_x, train_labels, val_x, val_labels, class_weights

In [6]:
train_x, train_labels, val_x, val_labels, class_weights = getSamples(1, 1)

getting ZA
getting EG
getting TW
getting JP
getting DK
getting FI
getting US
getting CA
getting AU
getting NZ
getting BE
getting CO


### Fit model

In [7]:
enc.categories_

[array(['AU', 'BE', 'CA', 'CO', 'DK', 'EG', 'FI', 'JP', 'NZ', 'TW', 'US',
        'ZA'], dtype='<U2')]

In [8]:
model = keras.Sequential()
model.add(LSTM(32, 
               input_shape=(train_x.shape[1], train_x.shape[2]), 
               return_sequences = True
              ))
model.add(BatchNormalization())
model.add(LSTM(64, 
               input_shape=(train_x.shape[1], train_x.shape[2]), 
               return_sequences = False,
               dropout = .5,
              go_backwards = True))
#model.add(BatchNormalization())
#model.add(LSTM(64, 
#               input_shape=(train_x.shape[1], train_x.shape[2]),
#               dropout = .5
#              ))
model.add(BatchNormalization())
model.add(Dropout(.5))
#model.add(Dense(64, activation = "relu"))
#model.add(BatchNormalization())
#model.add(Dropout(.5))
model.add(Dense(len(enc.categories_[0]), activation= "softmax"))
adam = keras.optimizers.Adam(lr=0.001)
model.compile(loss = "categorical_crossentropy", optimizer= adam, metrics=["acc"])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 300, 32)           7296      
_________________________________________________________________
batch_normalization (BatchNo (None, 300, 32)           128       
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                24832     
_________________________________________________________________
batch_normalization_1 (Batch (None, 64)                256       
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 12)                780       
Total params: 33,292
Trainable params: 33,100
Non-trainable params: 192
__________________________________________________

In [ ]:
log_dir = os.path.join(
    "logs",
    "fit",
    "32_64_Dropout_5000sample"
)
train_n = 5000
val_n = 1000
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
epochs = 25
iterations = 10
learn_rate = 0.001
for i in range(iterations):
    adam = keras.optimizers.Adam(lr=learn_rate)
    model.compile(loss = "categorical_crossentropy", optimizer= adam, metrics=["acc"])
    train_x, train_labels, val_x, val_labels, class_weights = getSamples(train_n, val_n)
    print(np.sum(train_labels, axis = 0))
    model.fit(train_x, train_labels,
              epochs = i * epochs + epochs, 
              initial_epoch = i * epochs,
              shuffle = True,
              validation_data = (val_x, val_labels),
              batch_size = 1024,
              class_weight = class_weights,
             callbacks=[tensorboard_callback],
             verbose = 1)
    learn_rate = learn_rate/2
    if i % 1 == 0:
        preds = model.predict(val_x, batch_size = 1024, verbose = 1)
     #   print(np.sum(train_labels, axis = 0))
        plt.imshow(
            confusion_matrix(
                enc.inverse_transform(preds), 
                enc.inverse_transform(val_labels), 
               # normalize = "all"
            )
        )
        plt.pause(.5)
        plt.show()
        preds = model.predict(train_x, batch_size = 1024, verbose = 1)
        plt.imshow(
            confusion_matrix(
                enc.inverse_transform(preds), 
                enc.inverse_transform(train_labels), 
            #    normalize = "all"
            )
        )
        plt.pause(.5)
        plt.show()

getting ZA
getting EG
getting TW
getting JP
getting DK
getting FI
getting US
getting CA
getting AU
getting NZ
getting BE
getting CO
[4350. 4296. 4300. 4351. 4242. 4292. 4315. 4363. 4316. 4428. 4336. 4403.]
Train on 51992 samples, validate on 10400 samples
Epoch 1/25
51992/51992 [==============================] - 158s 3ms/sample - loss: 3.2366 - acc: 0.0916 - val_loss: 2.4684 - val_acc: 0.1203
Epoch 2/25
51992/51992 [==============================] - 221s 4ms/sample - loss: 2.8523 - acc: 0.0992 - val_loss: 2.4594 - val_acc: 0.1280
Epoch 3/25
51992/51992 [==============================] - 260s 5ms/sample - loss: 2.6841 - acc: 0.1056 - val_loss: 2.4509 - val_acc: 0.1328
Epoch 4/25
51992/51992 [==============================] - 320s 6ms/sample - loss: 2.5926 - acc: 0.1146 - val_loss: 2.4437 - val_acc: 0.1388
Epoch 5/25
51992/51992 [==============================] - 348s 7ms/sample - loss: 2.5240 - acc: 0.1203 - val_loss: 2.4346 - val_acc: 0.1397
Epoch 6/25
51992/51992 [====================